In [1]:
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

import numpy as np
import random
import sys
import tensorflow as tf
import cv2
import time

In [9]:
class Dataset:
    def __init__(self):
        mnist = tf.contrib.learn.datasets.load_dataset('mnist')
        self.train_data = mnist.train.images#already float32, don't need to transform
        self.train_labels = np.asarray(mnist.train.labels, dtype=np.int32)#transform datatype of label from unit8 to int32
        self.eval_data = mnist.test.images
        self.eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

        self.curr_training_step = 0
        self.curr_test_step = 0

    def get_train_set_size(self):
        return self.train_data.shape[0]

    def get_test_set_size(self):
        return self.eval_data.shape[0]

    def to_one_hot(self, X):
        one_hot = np.zeros((len(X), 10))
        one_hot[np.arange(len(X)), X] = 1
        return one_hot

    def next_batch_train(self, batch_size):
        X_train_bs = self.train_data[(self.curr_training_step * batch_size):(self.curr_training_step * batch_size + batch_size), :]
        Y_train_bs = self.train_labels[self.curr_training_step * batch_size:self.curr_training_step * batch_size + batch_size]

        self.curr_training_step += 1
        self.curr_training_step = self.curr_training_step if self.curr_training_step * batch_size < self.train_data.shape[0] else 0

        return X_train_bs, self.to_one_hot(Y_train_bs)

    def next_batch_test(self, batch_size):
        X_test_bs = self.eval_data[(self.curr_test_step * batch_size):(self.curr_test_step * batch_size + batch_size), :]
        Y_test_bs = self.eval_labels[self.curr_test_step * batch_size:self.curr_test_step * batch_size + batch_size]

        self.curr_test_step += 1
        self.curr_test_step = self.curr_test_step if self.curr_test_step * batch_size < self.eval_data.shape[0] else 0

        return X_test_bs, self.to_one_hot(Y_test_bs)

    def visualize_train_sample(self, idx):
        img = np.reshape(self.train_data[idx, :], (28, 28))
        cv2.imshow('Train Sample', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [16]:
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

tf.logging.set_verbosity(tf.logging.INFO)

class LeNet:
    def __init__(self, weights=None, sess=None):
        self.X = tf.placeholder(tf.float32, shape=[None, 28, 28, 1], name='X')
        self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        
        self.sess = sess

        self.conv_layers()
        self.fc_layer()

        self.probs = tf.nn.softmax(self.logits, name='softmax')

    def conv_layers(self):
        self.parameters = []
        images = self.X

        with tf.name_scope('conv1') as scope:
            kernel_shape = (3, 3, 1, 32)
            kernel = tf.Variable(np.random.normal(0, np.sqrt(2.0/np.sum(kernel_shape)), kernel_shape), name='weights', dtype=tf.float32)
            conv = tf.nn.conv2d(images, kernel, strides=[1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(np.zeros(shape=[32]), dtype=tf.float32, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv1 = tf.nn.relu(out, name=scope)
            self.dropout1 = tf.nn.dropout(self.conv1, keep_prob=self.keep_prob, name='dropout1')
            self.parameters += [kernel, biases]

        self.pool1 = tf.nn.max_pool(self.dropout1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')

        with tf.name_scope('conv2') as scope:
            kernel_shape = (3, 3, 32, 64)
            kernel = tf.Variable(np.random.normal(0, np.sqrt(2.0/np.sum(kernel_shape)), kernel_shape), name='weights', dtype=tf.float32)
            conv = tf.nn.conv2d(self.pool1, kernel, strides=[1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(np.zeros(shape=[64]), dtype=tf.float32, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2 = tf.nn.relu(out, name=scope)
            self.dropout2 = tf.nn.dropout(self.conv2, keep_prob=self.keep_prob, name='dropout2')
            self.parameters += [kernel, biases]

        self.pool2 = tf.nn.max_pool(self.dropout2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool2')

    def fc_layer(self):
        with tf.name_scope('fc1') as scope:
            shape = int(np.prod(self.pool2.get_shape()[1:]))
            fc1w_shape = (shape, 128)
            fc1w = tf.Variable(np.random.normal(0, np.sqrt(2.0/np.sum(fc1w_shape)), fc1w_shape), name='weights', dtype=tf.float32)
            fc1b = tf.Variable(np.ones(shape=[128]), dtype=tf.float32, name='biases')
            # print(self.pool2.get_shape())

            pool2_flat = tf.reshape(self.pool2, [-1, shape])

            # print(pool2_flat.shape)
            fc1l = tf.nn.bias_add(tf.matmul(pool2_flat, fc1w), fc1b)
            self.fc1 = tf.nn.relu(fc1l)
            self.dropout3 = tf.nn.dropout(self.fc1, keep_prob=self.keep_prob, name='dropout3')
            self.parameters += [fc1w, fc1b]

        with tf.name_scope('fc2') as scope:
            fc2w_shape = (128, 10)
            fc2w = tf.Variable(np.random.normal(0, np.sqrt(2.0/np.sum(fc2w_shape)), fc2w_shape), name='weights', dtype=tf.float32)
            fc2b = tf.Variable(np.ones(shape=[10]), dtype=tf.float32, name='biases')
            fc2l = tf.nn.bias_add(tf.matmul(self.dropout3, fc2w), fc2b)
            self.logits = tf.nn.relu(fc2l)
            self.parameters += [fc2w, fc2b]

    def load_weights(weights, sess):
        None

    def train(self, learning_rate, training_epochs, batch_size, keep_prob):
        self.dataset = Dataset()
        self.Y = tf.placeholder(tf.float32, shape=[None, 10], name='Y')
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(self.cost)

        self.sess.run(tf.global_variables_initializer())
        self.sess.run(tf.local_variables_initializer())

        print('Training...')
        weights = []

        for epoch in range(training_epochs):
            avg_cost = 0

            total_batch = int(self.dataset.get_train_set_size()/batch_size)
            for i in range(total_batch + 1):
                batch_xs, batch_ys = self.dataset.next_batch_train(batch_size)
                feed_dict = {
                    self.X: batch_xs.reshape([batch_xs.shape[0], 28, 28, 1]),
                    self.Y: batch_ys,
                    self.keep_prob: keep_prob
                }
                weights, c, _ = self.sess.run([self.parameters, self.cost, self.optimizer], feed_dict=feed_dict)
                avg_cost += c / total_batch
                
            print('Epoch:', '%02d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

        print('Training finished!')

        saver = tf.train.Saver()
        save_path = saver.save(self.sess, "/tmp/mnist_brain.ckpt")
        print("Trainned model is saved in file: %s" % save_path)

    def evaluate(self, batch_size, keep_prob):

        self.correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        N = self.dataset.get_test_set_size()
        print('test.size', N);
        correct_sample = 0
        for i in range(0, N, batch_size):
            batch_xs, batch_ys = self.dataset.next_batch_test(batch_size)

            N_batch = batch_xs.shape[0]

            feed_dict = {
                self.X: batch_xs.reshape([N_batch, 28, 28, 1]),
                self.Y: batch_ys,
                self.keep_prob: keep_prob
            }

            correct = self.sess.run(self.accuracy, feed_dict=feed_dict)
            correct_sample += correct * N_batch

        test_accuracy = correct_sample / N

        print("\nAccuracy Evaluates")
        print("-" * 30)
        print('Test Accuracy:', test_accuracy)

start = time.time()

sess = tf.Session()
lenet = LeNet(sess=sess, weights=None)

lenet.train(learning_rate=0.001, training_epochs=40, batch_size=1000, keep_prob=0.7)
print(time.strftime("%H:%M:%S", time.gmtime(time.time() - start)))

lenet.evaluate(batch_size=1000, keep_prob=0.7)

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
Training...
Epoch: 01 cost = 2.525377742
Epoch: 02 cost = 2.187140556
Epoch: 03 cost = 2.033181546
Epoch: 04 cost = 1.793702791
Epoch: 05 cost = 1.495445191
Epoch: 06 cost = 1.209689361
Epoch: 07 cost = 1.045863707
Epoch: 08 cost = 0.945034527
Epoch: 09 cost = 0.871758809
Epoch: 10 cost = 0.808018292
Epoch: 11 cost = 0.764217846
Epoch: 12 cost = 0.723358573
Epoch: 13 cost = 0.696367759
Epoch: 14 cost = 0.665631488
Epoch: 15 cost = 0.640987975
Epoch: 16 cost = 0.618492712
Epoch: 17 cost = 0.596949820
Epoch: 18 cost = 0.588280585
Epoch: 19 cost = 0.571549820
Epoch: 20 cost = 0.553694396
Epoch: 21 cost = 0.542154212
Epoch: 22 cost = 0.534017015
Epoch: 23 cost = 0.521516288
Epoch: 24 cost = 0.506690454
Epoch: 25 cost = 0.506198347
Epoch: 26 cost = 0.492842772
Epoch: 27 cost = 0.486572572
Epoch: 28 c

In [ ]:
# w = tf.Variable(np.random.normal(0, np.sqrt(2.0/np.sum(w_shape)), w_shape), name='weights')

In [3]:
import time

start = time.time()
a = np.array([1, 2, 3])
b = 1
a + b
time.strftime("%H:%M:%S", time.gmtime(time.time() - start))

'00:00:00'

In [15]:
tf.logging.set_verbosity(tf.logging.INFO)

class LeNet():
    def __init__(self, weights=None, sess=None, log=True):
        self.X = tf.placeholder(tf.float32, [None, 28, 28, 1], name='X')
        self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        self.log = log
        self.sess = sess

        self.conv_layers()
        self.fc_layers()

        self.probs = tf.nn.softmax(self.logits, name='softmax')

    def conv_layers(self):
        self.parameters = []
        images = self.X

        # Layer 1: Conv
        with tf.name_scope('conv1') as scope:
            kernel = tf.Variable(tf.random_normal([3, 3, 1, 32], dtype=tf.float32, stddev=1e-1, name='weights'))
            conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[32], dtype=tf.float32), trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # Layer 1: Pooling
        self.pool1 = tf.nn.max_pool(self.conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name='pool1')

        # Layer 2: Conv
        with tf.name_scope('conv2') as scope:
            kernel = tf.Variable(tf.random_normal([3, 3, 32, 64], dtype=tf.float32, stddev=1e-1, name='weights'))
            conv = tf.nn.conv2d(self.pool1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32), trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # Layer 2: Pooling
        self.pool2 = tf.nn.max_pool(self.conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name='pool2')

    def fc_layers(self):
    # fc1
        with tf.name_scope('fc1') as scope:
            shape = int(np.prod(self.pool2.get_shape()[1:]))
            fc1w = tf.Variable(tf.random_normal([shape, 128], dtype=tf.float32, stddev=1e-1), name='weights')
            fc1b = tf.Variable(tf.constant(1.0, shape=[128], dtype=tf.float32), trainable=True, name='biases')
            pool2_flat = tf.reshape(self.pool2, [-1, shape])
            fc1l = tf.nn.bias_add(tf.matmul(pool2_flat, fc1w), fc1b)
            self.fc1 = tf.nn.relu(fc1l)
            self.dropout1 = tf.nn.dropout(self.fc1, keep_prob=self.keep_prob, name='dropout1')
            self.parameters += [fc1w, fc1b]

        # fc2
        with tf.name_scope('fc2') as scope:
            fc2w = tf.Variable(tf.random_normal([128, 10], dtype=tf.float32, stddev=1e-1), name='weights')
            fc2b = tf.Variable(tf.constant(1.0, shape=[10], dtype=tf.float32), trainable=True, name='biases')
            fc2l = tf.nn.bias_add(tf.matmul(self.dropout1, fc2w), fc2b)
            self.logits = tf.nn.relu(fc2l)
            self.parameters += [fc2w, fc2b]

    def train(self, learning_rate, training_epochs, batch_size, keep_prob):
        # Load dataset for training and testing
        self.dataset = DataSet()

        # Define size of output
        self.Y = tf.placeholder(tf.float32, [None, 10], name='Y')
        # Define cost function
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
        # Define optimization method
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

        # Start logger
        if self.log:
            tf.summary.scalar('cost', self.cost)
            self.merged = tf.summary.merge_all()
            self.train_writer = tf.summary.FileWriter('./log_train', self.sess.graph)

        self.sess.run(tf.global_variables_initializer())
        self.sess.run(tf.local_variables_initializer())

        print('Training...')
        weights = []
        # For each epoch, feed training data and perform updating parameters
        for epoch in range(training_epochs):
            avg_cost = 0
            # Number of batches = size of training set / batch_size
            total_batch = int(self.dataset.get_train_set_size() / batch_size)

            # For each batch 
            for i in range(total_batch + 1):
                # Get next batch to feed to the network
                batch_xs, batch_ys = self.dataset.next_batch(batch_size)
                feed_dict = {
                    self.X: batch_xs.reshape([batch_xs.shape[0], 28, 28, 1]),
                    self.Y: batch_ys,
                    self.keep_prob: keep_prob
                }

                weights, summary, c, _ = self.sess.run([self.parameters, self.merged, self.cost, self.optimizer], feed_dict=feed_dict)
                avg_cost += c / total_batch

            if self.log:
                self.train_writer.add_summary(summary, epoch + 1)

            print('Epoch:', '%02d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

        print('Training finished!')

        saver = tf.train.Saver()
        save_path = saver.save(self.sess, "/tmp/mnist_brain.ckpt")
        print("Trainned model is saved in file: %s" % save_path)

    def evaluate(self, batch_size, keep_prob):

        self.correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        N = self.dataset.get_test_set_size()
        print('test.size', N);
        correct_sample = 0
        for i in range(0, N, batch_size):
            batch_xs, batch_ys = self.dataset.next_batch_test(batch_size)

            N_batch = batch_xs.shape[0]

            feed_dict = {
                self.X: batch_xs.reshape([N_batch, 28, 28, 1]),
                self.Y: batch_ys,
                self.keep_prob: keep_prob
            }

            correct = self.sess.run(self.accuracy, feed_dict=feed_dict)
            correct_sample += correct * N_batch

        test_accuracy = correct_sample / N

        print("\nAccuracy Evaluates")
        print("-" * 30)
        print('Test Accuracy:', test_accuracy)

sess = tf.Session()
lenet = LeNet(sess=sess, weights=None)

lenet.train(learning_rate=0.001, training_epochs=40, batch_size=1000, keep_prob=0.7)
lenet.evaluate(batch_size=1000, keep_prob=0.7)

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
Training...


InvalidArgumentError: You must feed a value for placeholder tensor 'X' with dtype float and shape [?,28,28,1]
	 [[Node: X = Placeholder[dtype=DT_FLOAT, shape=[?,28,28,1], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: conv2_3/biases/_27 = _Recv[_start_time=0, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_529_conv2_3/biases", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'X', defined at:
  File "D:\Anaconda\envs\tsf36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Anaconda\envs\tsf36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Anaconda\envs\tsf36\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "D:\Anaconda\envs\tsf36\lib\site-packages\traitlets\config\app

In [21]:
tmp = np.array([[1, 2, 3, 4]])
tmp + np.array([1, 2, 3, 4])

array([[2, 4, 6, 8]])

In [5]:
tmp.shape

(2, 3)

In [13]:
g1 = tf.Graph()
with g1.as_default():
    with tf.Session() as sess:
        A = tf.constant([5, 7], dtype=tf.int32, name='A')
        x = tf.placeholder(tf.int32, name='x')
        b = tf.constant([3, 4], dtype=tf.int32, name='b')

        y = A * x + b

        print(sess.run(y, feed_dict={'x:0': [10, 100]}))

[ 53 704]


In [2]:
A = tf.constant([4], tf.int32, name='A')
B = tf.constant([4], tf.int32, name='B')
C = tf.constant([4], tf.int32, name='C')

x = tf.placeholder(tf.int32, name='x')

# y = Ax^2 + Bx + C
with tf.name_scope('Equation1'):
    Ax2_1 = tf.multiply(A, tf.pow(x,2), name="Ax2_1")
    Bx = tf.multiply(A,x, name="Bx")
    y1 = tf.add_n([Ax2_1, Bx, C], name='y1')

# y = Ax^2 + Bx^2
with tf.name_scope('Equation2'):
    Ax2_2 = tf.multiply(A, tf.pow(x,2),name='Ax2_2')
    Bx2 = tf.multiply(B, tf.pow(x,2),name='Bx2')
    y2 = tf.add_n([Ax2_2,Bx2],name='y2')

with tf.name_scope('conv'):
    y = y1 + y2

with tf.Session() as sess:
    print(sess.run(y, feed_dict={'x:0': [10]}))
    
writer = tf.summary.FileWriter('./name_scope/exp1', sess.graph)
writer.close()

[1244]


In [11]:
?tf.placeholder()

Signature: tf.placeholder(dtype, shape=None, name=None)
Docstring:
Inserts a placeholder for a tensor that will be always fed.

**Important**: This tensor will produce an error if evaluated. Its value must
be fed using the `feed_dict` optional argument to `Session.run()`,
`Tensor.eval()`, or `Operation.run()`.

For example:

```python
x = tf.placeholder(tf.float32, shape=(1024, 1024))
y = tf.matmul(x, x)

with tf.Session() as sess:
  print(sess.run(y))  # ERROR: will fail because x was not fed.

  rand_array = np.random.rand(1024, 1024)
  print(sess.run(y, feed_dict={x: rand_array}))  # Will succeed.
```

@compatibility{eager} Placeholders are not compatible with eager execution.

Args:
  dtype: The type of elements in the tensor to be fed.
  shape: The shape of the tensor to be fed (optional). If the shape is not
    specified, you can feed a tensor of any shape.
  name: A name for the operation (optional).

Returns:
  A `Tensor` that may be used as a handle for feeding a value, but n

In [8]:
a = []
a += [2, 3]
a += [4, 6]
a

[2, 3, 4, 6]

In [21]:
X = np.array([2, 3, 4])
tmp = np.zeros((3, 5))
tmp[np.arange(3), X] = 1
tmp

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [14]:
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)

In [17]:
train_labels[:5]

array([7, 3, 4, 6, 1])

In [16]:
mnist.train.labels[:5]

array([7, 3, 4, 6, 1], dtype=uint8)

In [12]:
?np.array

Docstring:
array(object, dtype=None, copy=True, order='K', subok=False, ndmin=0)

Create an array.

Parameters
----------
object : array_like
    An array, any object exposing the array interface, an object whose
    __array__ method returns an array, or any (nested) sequence.
dtype : data-type, optional
    The desired data-type for the array.  If not given, then the type will
    be determined as the minimum type required to hold the objects in the
    sequence.  This argument can only be used to 'upcast' the array.  For
    downcasting, use the .astype(t) method.
copy : bool, optional
    If true (default), then the object is copied.  Otherwise, a copy will
    only be made if __array__ returns a copy, if obj is a nested sequence,
    or if a copy is needed to satisfy any of the other requirements
    (`dtype`, `order`, etc.).
order : {'K', 'A', 'C', 'F'}, optional
    Specify the memory layout of the array. If object is not an array, the
    newly created array will be in C order (